In [ ]:
import sys 
import os
sys.path.append(os.path.abspath("../../"))
from ampel.ztf.dev.ZTFAlert import ZTFAlert
from ampel.contrib.hu.t2.T2BrightSNProb import T2BrightSNProb
from ampel.log.AmpelLogger import AmpelLogger
import requests
from requests.auth import HTTPBasicAuth
from ampel.ztf.util import ZTFIdMapper
from ampel.ztf.ingest.ZiDataPointShaper import ZiDataPointShaper
from ampel.content.T1Document import T1Document
from ampel.view.LightCurve import LightCurve

# Running SNGuess over a ZTF candidate's alerts

In this notebook we will query the DESY archive for the alerts of a ZTF candidate, and then run SNGuess over it in order to obtain a relevance score.

## Steps

First, we enter the connection parameters for the DESY archive. In our case, this is a token that can be obtained in https://ampel.zeuthen.desy.de/live/dashboard/tokens .

If your GitHub user does not have access to the ZTF or AMPEL organizations, please send an e-mail to

In [ ]:
token = # Add token string

Next, we enter the ZTF identifier of the candidate we want to analyze.

In [ ]:
name = 'ZTF20abyfpze'

We initialize the relevant AMPEL T2 units.

In [ ]:
logger = AmpelLogger.get_logger()
t2snguess = T2BrightSNProb( logger=logger)
t2snguess.post_init()
shaper = ZiDataPointShaper(logger=logger)

We set the DESY archive endpoint url and request the alerts of the candidate using our connection token.

In [ ]:
endpoint = "https://ampel.zeuthen.desy.de/api/ztf/archive/object/{}/alerts?with_history=true&with_cutouts=false".format(name)
headers = {'Authorization': f"bearer {token}"}

In [ ]:
response = requests.get(endpoint, headers=headers)
response.raise_for_status()

In [ ]:
alerts = response.json()
print("Found {} alerts for {}".format(len(alerts), name))

Finally, we iterate over the alerts and run SNGuess over them in order to obtain a relevance score and a boolean that indicates whether its candidate is likely to be relevant for follow up observations or not.

In [ ]:
summary = []
for alert in alerts:
    
    # Create standardized LightCurve object
    pps = [alert['candidate']]
    pps.extend( [prv_cand for prv_cand in alert['prv_candidates'] ] )
    
    # The following loop is _likely_ due to an inconsistency in the alert archive with the shaper
    # and can hopefully be removed soon
    for pp in pps:
        if "magpsf" in pp.keys():
            pp["candid"] = 999

    stockId = ZTFIdMapper.to_ampel_id(name)
    dps = shaper.process( pps, stockId)
    t1d = T1Document(stock=stockId, link=0)
    lc = LightCurve.build(t1d, dps)
    
    # Content
    jds = lc.get_values("jd")
    if jds is None:
        continue
    t2out = t2snguess.process(lc)
    print(t2out)
    
    if t2out['success']:
        summary.append( [max(jds),len(jds),t2out['success'], t2out['SNGuess'], t2out['SNGuessBool'] ])
    else:
        summary.append( [max(jds),len(jds),t2out['success'] ])

We display the results.

In [ ]:
summary